<a href="https://colab.research.google.com/github/dhruv20047228/LLM/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install uv

In [ ]:
!uv pip install -r https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/requirements.txt --system

# **Tokenizing the Text**

In [ ]:
from importlib.metadata import version
print("torch version:", version("torch"))
print("tiktoken version:",version("tiktoken"))

In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
  url = ("https://raw.githubusercontent.com/rasbt/"
    "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
    "the-verdict.txt")
  file_path = "the-verdict.txt"
  urllib.request.urlretrieve(url, file_path)


In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])


**Developing Tokenizer for LLM**

In [ ]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text) #this converts words into token and also their in between spacing

print(result)

In [ ]:
result = re.split(r'([,.]|\s)', text) #this converts spacing and commas seperately into tokens
print(result)

In [ ]:
result = [item for item in result if item.strip()] #strped the whitespaces from the tokens
print(result)

In [ ]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(result)
print(len(preprocessed))

# **Converting tokens into token IDs**

In [ ]:
all_words = sorted(set(preprocessed)) #sorting all the tokens alphabetically with algorithm
vocab_size = len(all_words)
print(vocab_size)

In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
  print(item)
  if i >= 50:
    break

In [ ]:
import re

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [ ]:
#Encoding the text
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
        Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)


In [ ]:
tokenizer.decode(ids) #using ids variable for decoding

In [ ]:
tokenizer.decode(tokenizer.encode(text)) ##explicitly decoding

# **Adding special context tokens**

In [ ]:
all_tokens = sorted(list(preprocessed))
all_tokens.extend(["<|endoftext|>", "<|unk|>"]) # endoftext is used for concatenation of two unknown text, unk used for defining unknown text which might be context token or else

vocab = {token:integer for integer,token in enumerate(all_tokens)}
len(vocab.items())

In [ ]:
for i,item in enumerate(list(vocab.items())[-5:]):
  print(item)

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = "<|endoftext|>".join((text1, text2))

print(text)

In [ ]:
tokenizer.encode(text)

In [ ]:
tokenizer.decode(tokenizer.encode(text))

#**Bytepair encoding**

In [ ]:
!pip install tiktoken

In [ ]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")

In [ ]:

text = (
    "Hello, do you like tea? <|endoftoken|> In the sunlit terraces"
    "of someunkownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

In [ ]:
strings = tokenizer.decode(integers)
print(strings)

#**Data sampling with a sliding window**

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

In [ ]:
enc_sample = enc_text[50:] #sliding window algo for text prediction

In [ ]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

In [ ]:
#one by one token id prediction
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(context, "---->", desired)

In [ ]:
#one by one token prediction
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

#**Data Loader for Dataset**

In [ ]:
import torch
print("Pytorch version:", torch.__version__)

In [ ]:
#Creating dataset
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self, txt, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
    assert len(token_ids) > max_length, "Number of tokenized inputs must be equal to max_length+1"

    #using sliding window algorithm for word traversal letter by letter
    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i:i + max_length]
      target_chunk = token_ids[i + 1: i + max_length + 1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [ ]:
#Dataloader
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    #initialisation of tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")

    #Creating dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    #Creating Dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader



In [ ]:
#testing dataloader with batch size of 1 for LLM with context size of 4
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False #stride helps to not overlap tokens
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

In [ ]:
second_batch = next(data_iter)
print(second_batch)

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

#**Creating token embeddings**

In [ ]:
input_ids = torch.tensor([2,3,5,1])


In [ ]:
#Creation of embeddings
vocab_size = 6
output_dim = 3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

In [ ]:
print(embedding_layer(torch.tensor([3])))

In [ ]:
print(embedding_layer(input_ids))

#**Encoding word postions**